In [1]:
using Random
using DataStructures

In [2]:
# returns dup char --> num instances dictionary, num of duplicate chars  
##### returns 2 x N array (N = num dup char) of duplicate char & num instances, num of duplicate chars

function find_dups_in_str(str::String)
    alphabet = Set{Char}() 
    duplicates = OrderedDict{Char, Int}()  # duplicate char --> num instances 

    for char in str
        char = lowercase(char)   
        if char in alphabet 
            if char in keys(duplicates)
                duplicates[char] += 1
            else 
                duplicates[char] = 2 
            end 
        else 
            push!(alphabet, char)
        end 
    end 

    return duplicates, length(duplicates) 
end 

find_dups_in_str (generic function with 1 method)

In [3]:
function generate_map(num_dups::Int, dup_to_num_instances::OrderedDict{Char, Int})
    map = Array{Int}(undef, num_dups)
    
    idx = 1
    # for each position of the map (each gene w >1 instances)
    for (dup_char, num_instances) in pairs(dup_to_num_instances)
        # an integer value is selcted uniformly from the interval [0, occ(α, S)! -1] 
        mapidx_for_dup_char = rand(1:factorial(num_instances))
        
        map[idx] = mapidx_for_dup_char
        idx += 1
    end 
    
    return map
end 

generate_map (generic function with 1 method)

In [8]:
# DP implementation of factorial
function Base.factorial(n::Int, factorials::Vector{Int})
    if n == 1 || n == 0 
        return 1
    end 

    if 1 <= n <= length(factorials) 
        return factorials[n]
    else 
        fact =  n * factorial(n - 1, factorials)
        push!(factorials, fact)
        return fact
    end 
end

# returns kth lexicogrpahical order of the first n ints 
# k ε [1,n!]
function nth_lex_permutation(n::Int, k::Int)
    k -= 1
    factorials = Vector{Int}()
    
    numbers = collect(1:n)
    permutation = Vector{Int}()
    
    for i in 1:n
        fact = factorial(n - i, factorials)
        index = div(k, fact)
        k %= fact
        push!(permutation, numbers[index+1])
        splice!(numbers, index+1)
    end
    
    return permutation
end

# n = 3  # num duplicates 
# k = 1  #  kth lexicogrpahical order of the first n ints [1:n!]

# perm = nth_lex_permutation(n, k)
# println(perm)

[1, 2, 3]


In [34]:
function new_char(alphabet::Set{Char})
    while true 
        c = Char(rand('a':'z'))
        if c ∉ alphabet 
            push!(alphabet, c)
            return c
        end 
    end 
end 


function deduplicate_genome(map::Array{Int}, dup_genome::String, dupchar_to_multiplicity::OrderedDict{Char, Int}, dupchar_to_unique_char::Dict{Char, Vector{Char}})
    if dupchar_to_unique_char == Dict{Char, Vector{Char}}()  # passed in target genome; need to assign dup chars to unique chars
        alphabet = Set(dup_genome)
        
        idx = 1
        # create mapping from duplicated chars (multiplicity k) --> [array of k unique chars], idx = idx of duplicate used in permutation 
        for (dupchar, mult) in pairs(dupchar_to_multiplicity)
            unique_chars = [dupchar]
            for i in 2:dupchar_to_multiplicity[dupchar]
                c = new_char(alphabet)
                push!(unique_chars, c)
            end     
            dupchar_to_unique_char[dupchar] = unique_chars
            
            idx += 1
        end 
        println(dupchar_to_unique_char)
    end 

    idx = 1
    for (dupchar, mult) in pairs(dupchar_to_multiplicity)
        dup_perm = nth_lex_permutation(mult, map[idx])
        
        println(dupchar, dup_perm)

        

        idx += 1
    end 

    # for k in dupchar_to_multiplicity 
    #     permutation = nth_lex_permutation 
    # end 
    # return dedup_genome 
end 


P = "abbAac"
P_dup_to_num_instances, P_num_dups = find_dups_in_str(P)
P_map = generate_map(P_num_dups, P_dup_to_num_instances)
dupchar_to_unique_char = Dict{Char, Vector{Char}}()

deduplicate_genome(P_map, P, P_dup_to_num_instances, dupchar_to_unique_char)

Dict('a' => ['a', 'p', 'w'], 'b' => ['b', 'z'])
b[1, 2]
a[3, 2, 1]


In [ ]:
# mapping S --> T, generates r random maps (RM)

function randommap(S::String, P::String, num_maps::Int)
    println("SRC ", S, " --> TARGET ", P, "\n")

    ## GENERATE MAPS  

    S_dupchar_to_multiplicity, S_num_dups = find_dups_in_str(S)
    P_dupchar_to_multiplicity, P_num_dups = find_dups_in_str(P)

    printstyled("target", color=:magenta)
    println("\ndup genes --> multiplicity")
    println(P_dupchar_to_multiplicity, "\n")

    # arbitrary map p for target string 
    P_map = ones(Int, P_num_dups)
    println("target map")
    print(P_map, "\n")
    

    printstyled("\nsource", color=:magenta)
    println("\ndup genes --> multiplicity")
    println(S_dupchar_to_multiplicity, "\n")

    S_M = Array{Array{Int}}(undef, num_maps)  # TODO does copy elements or point to them? 

    # r RM of src str S are generated and stored in a set S_M 
    println("source maps")
    for i in 1:num_maps
        s_map = generate_map(S_num_dups, S_dupchar_to_multiplicity)
        println(s_map)
        push!(S_M, s_map)
    end 

    ## DEDUPICATION 
    dup_char_to_unique_char = Dict{Char:Vector{Char}}()
    
    # use maps to deduplicate genomes 
    # dedup_P = deduplicate_genome(P_map, P, P_dup_to_num_instances, dup_char_to_unique_char)

    # lehmer code to convert string S and P --> deduplicated S and P using the generated maps  
    # nth_lex_permutation(n, k)
    
        # for each elem in each map convert int n --> nth permutation of n ints in lexicographical order 

   
        
    # intiially the target string is mapped into permutation Pp using an arbitrary map p 
    
    # for each map m in M 
        # distance between permutation Sm and Pp is computed using estimator algo 

    #return shortest map iwth shortest dist between permutations 
end 


src = "aabc"
target = "abAac"
num_maps = 3

randommap(src, target, num_maps)